In [1]:
import os
import zipfile
from concurrent.futures import ThreadPoolExecutor


In [2]:
import os
import zipfile

def list_csv_files_in_zips(folder_path):
    """
    Lists all CSV files within multiple ZIP archives in a folder.

    Parameters:
    folder_path (str): The path to the folder containing ZIP files.

    Returns:
    dict: A dictionary with ZIP file names as keys and a list of CSV file names as values.
    """
    csv_files = {}
    
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.zip'):
                zip_path = os.path.join(root, file)
                try:
                    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                        csv_list = [name for name in zip_ref.namelist() if name.endswith('.csv')]
                        if csv_list:
                            csv_files[file] = csv_list
                except Exception as e:
                    csv_files[file] = f"An error occurred: {e}"
    
    return csv_files

# Example usage
folder_path = 'path/to/folder'
csv_files = list_csv_files_in_zips(folder_path)
for zip_file, csv_list in csv_files.items():
    print(f"CSV files in {zip_file}:")
    for csv_file in csv_list:
        print(f" - {csv_file}")


In [3]:
folder_path = '/Users/thusondube/Downloads/itineraries_csv'
csv_files = list_csv_files_in_zips(folder_path)
for zip_file, csv_list in csv_files.items():
    print(f"CSV files in {zip_file}:")
    for csv_file in csv_list:
        print(f" - {csv_file}")

CSV files in OAK_itineraries_cq.zip:
 - OAK_itineraries_cq.csv
CSV files in OAK_itineraries_an.zip:
 - OAK_itineraries_an.csv
CSV files in OAK_itineraries_bu.zip:
 - OAK_itineraries_bu.csv
CSV files in OAK_itineraries_cf.zip:
 - OAK_itineraries_cf.csv
CSV files in OAK_itineraries_bb.zip:
 - OAK_itineraries_bb.csv
CSV files in OAK_itineraries_ay.zip:
 - OAK_itineraries_ay.csv
CSV files in OAK_itineraries_or.zip:
 - OAK_itineraries_or.csv
CSV files in OAK_itineraries_li.zip:
 - OAK_itineraries_li.csv
CSV files in OAK_itineraries_nv.zip:
 - OAK_itineraries_nv.csv
CSV files in OAK_itineraries_mm.zip:
 - OAK_itineraries_mm.csv
CSV files in OAK_itineraries_oe.zip:
 - OAK_itineraries_oe.csv
CSV files in OAK_itineraries_mz.zip:
 - OAK_itineraries_mz.csv
CSV files in OAK_itineraries_na.zip:
 - OAK_itineraries_na.csv
CSV files in OAK_itineraries_xl.zip:
 - OAK_itineraries_xl.csv
CSV files in OAK_itineraries_yh.zip:
 - OAK_itineraries_yh.csv
CSV files in OAK_itineraries_zs.zip:
 - OAK_itineraries

In [ ]:
import os
import zipfile
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def list_csv_files_in_zips(folder_path):
    """
    Lists all CSV files within multiple ZIP archives in a folder.

    Parameters:
    folder_path (str): The path to the folder containing ZIP files.

    Returns:
    dict: A dictionary with ZIP file names as keys and a list of CSV file names as values.
    """
    csv_files = {}
    
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.zip'):
                zip_path = os.path.join(root, file)
                try:
                    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                        csv_list = [name for name in zip_ref.namelist() if name.endswith('.csv')]
                        if csv_list:
                            csv_files[zip_path] = csv_list
                except Exception as e:
                    csv_files[zip_path] = f"An error occurred: {e}"
    
    return csv_files

def read_csv_from_zip(zip_path, csv_file):
    """
    Reads a CSV file from a ZIP archive and returns it as a DataFrame.

    Parameters:
    zip_path (str): The path to the ZIP file.
    csv_file (str): The name of the CSV file within the ZIP archive.

    Returns:
    pd.DataFrame: A DataFrame containing the data from the CSV file.
    """
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            with zip_ref.open(csv_file) as f:
                return pd.read_csv(f)
    except Exception as e:
        print(f"An error occurred while reading {csv_file} from {zip_path}: {e}")
        return pd.DataFrame()

def merge_csv_files_from_zips_to_dataframe(folder_path):
    """
    Merges all CSV files within multiple ZIP archives in a folder into one DataFrame.

    Parameters:
    folder_path (str): The path to the folder containing ZIP files.

    Returns:
    pd.DataFrame: A DataFrame containing the merged data from all CSV files.
    """
    csv_files = list_csv_files_in_zips(folder_path)
    merged_df = pd.DataFrame()
    
    with ThreadPoolExecutor() as executor:
        futures = []
        for zip_path, csv_list in csv_files.items():
            if isinstance(csv_list, list):
                for csv_file in csv_list:
                    futures.append(executor.submit(read_csv_from_zip, zip_path, csv_file))
        
        for future in futures:
            df = future.result()
            merged_df = pd.concat([merged_df, df], ignore_index=True)
    
    return merged_df

# Example usage
folder_path = '/Users/thusondube/Downloads/itineraries_csv'
merged_df = merge_csv_files_from_zips_to_dataframe(folder_path)




In [ ]:
folder_path = '/Users/thusondube/Downloads/itineraries_csv'
merged_df = merge_csv_files_from_zips_to_dataframe(folder_path)


In [2]:
import os
import zipfile
import pandas as pd

def list_csv_files_in_zips(folder_path, max_folders=2):
    """
    Lists all CSV files within the first two ZIP archives in a folder.

    Parameters:
    folder_path (str): The path to the folder containing ZIP files.
    max_folders (int): The maximum number of ZIP folders to process.

    Returns:
    dict: A dictionary with ZIP file names as keys and a list of CSV file names as values.
    """
    csv_files = {}
    folder_count = 0
    
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.zip'):
                zip_path = os.path.join(root, file)
                try:
                    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                        csv_list = [name for name in zip_ref.namelist() if name.endswith('.csv')]
                        if csv_list:
                            csv_files[zip_path] = csv_list
                            folder_count += 1
                            if folder_count >= max_folders:
                                return csv_files
                except Exception as e:
                    csv_files[zip_path] = f"An error occurred: {e}"
    
    return csv_files

def merge_csv_files_from_zips_to_dataframe(folder_path, max_folders=2):
    """
    Merges all CSV files within the first two ZIP archives in a folder into one DataFrame.

    Parameters:
    folder_path (str): The path to the folder containing ZIP files.
    max_folders (int): The maximum number of ZIP folders to process.

    Returns:
    pd.DataFrame: A DataFrame containing the merged data from all CSV files.
    """
    csv_files = list_csv_files_in_zips(folder_path, max_folders)
    merged_df = pd.DataFrame()
    
    for zip_path, csv_list in csv_files.items():
        if isinstance(csv_list, list):
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                for csv_file in csv_list:
                    with zip_ref.open(csv_file) as f:
                        df = pd.read_csv(f)
                        merged_df = pd.concat([merged_df, df], ignore_index=True)
    
    return merged_df

# Example usage
folder_path = '/Users/thusondube/Downloads/itineraries_csv'
merged_df = merge_csv_files_from_zips_to_dataframe(folder_path)
merged_df.head()


,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,e1b137527b9175d7d930c3af82e70ae0,2022-04-19,2022-05-20,OAK,ATL,PT7H52M,False,False,False,103.98,...,1653107460||1653126600,2022-05-20T22:31:00.000-06:00||2022-05-21T05:5...,DEN||ATL,OAK||DEN,Frontier Airlines||Frontier Airlines,F9||F9,||Airbus A320,9180||10620,943||1207,coach||coach
1,d813ebd107e3fa700206c0d96015da7a,2022-04-19,2022-05-20,OAK,ATL,PT6H15M,False,False,False,216.58,...,1653067080||1653084660,2022-05-20T10:18:00.000-07:00||2022-05-20T18:1...,LAX||ATL,OAK||LAX,Spirit Airlines||Spirit Airlines,NK||NK,||AIRBUS INDUSTRIE A320 SHARKLETS,4920||15600,None||None,coach||coach
2,e8ece5ad6f5962c696e06e031fc2a24a,2022-04-19,2022-05-20,OAK,ATL,PT9H6M,False,False,False,216.58,...,1653056820||1653084660,2022-05-20T07:27:00.000-07:00||2022-05-20T18:1...,LAX||ATL,OAK||LAX,Spirit Airlines||Spirit Airlines,NK||NK,AIRBUS INDUSTRIE A320 SHARKLETS||AIRBUS INDUST...,4920||15600,None||None,coach||coach
3,c004a54681335100f326c9613b3c9448,2022-04-19,2022-05-20,OAK,ATL,PT6H17M,False,False,False,237.58,...,1653110940||1653127980,2022-05-20T22:29:00.000-07:00||2022-05-21T06:1...,LAS||ATL,OAK||LAS,Spirit Airlines||Spirit Airlines,NK||NK,AIRBUS INDUSTRIE A320 SHARKLETS||Airbus A319,5580||13980,None||None,coach||coach
4,4a42bbf77211b4afa7b9e14005949120,2022-04-19,2022-05-20,OAK,ATL,PT14H12M,False,False,False,307.21,...,1653115560||1653159180,2022-05-20T23:46:00.000-07:00||2022-05-21T14:5...,SEA||ATL,OAK||SEA,Alaska Airlines||Alaska Airlines,AS||AS,Boeing 737-900||Boeing 737-900,7500||17580,672||2178,coach||coach


In [ ]:
import os
import zipfile
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def list_csv_files_in_zips(folder_path):
    """
    Lists all CSV files within multiple ZIP archives in a folder.

    Parameters:
    folder_path (str): The path to the folder containing ZIP files.

    Returns:
    dict: A dictionary with ZIP file names as keys and a list of CSV file names as values.
    """
    csv_files = {}
    
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.zip'):
                zip_path = os.path.join(root, file)
                try:
                    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                        csv_list = [name for name in zip_ref.namelist() if name.endswith('.csv')]
                        if csv_list:
                            csv_files[zip_path] = csv_list
                except Exception as e:
                    csv_files[zip_path] = f"An error occurred: {e}"
    
    return csv_files

def read_csv_from_zip(zip_path, csv_file):
    """
    Reads a CSV file from a ZIP archive and returns it as a DataFrame.

    Parameters:
    zip_path (str): The path to the ZIP file.
    csv_file (str): The name of the CSV file within the ZIP archive.

    Returns:
    pd.DataFrame: A DataFrame containing the data from the CSV file.
    """
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            with zip_ref.open(csv_file) as f:
                return pd.read_csv(f)
    except Exception as e:
        print(f"An error occurred while reading {csv_file} from {zip_path}: {e}")
        return pd.DataFrame()

def merge_csv_files_from_zips_to_parquet(folder_path, output_file):
    """
    Merges all CSV files within multiple ZIP archives in a folder into one Parquet file.

    Parameters:
    folder_path (str): The path to the folder containing ZIP files.
    output_file (str): The path to the output Parquet file.
    """
    csv_files = list_csv_files_in_zips(folder_path)
    merged_df = pd.DataFrame()
    
    with ThreadPoolExecutor() as executor:
        futures = []
        for zip_path, csv_list in csv_files.items():
            if isinstance(csv_list, list):
                for csv_file in csv_list:
                    futures.append(executor.submit(read_csv_from_zip, zip_path, csv_file))
        
        for future in futures:
            df = future.result()
            merged_df = pd.concat([merged_df, df], ignore_index=True)
    
    merged_df.to_parquet(output_file, engine='pyarrow')
    print(f"All CSV files have been merged and saved to {output_file}")

# Example usage
folder_path = '/Users/thusondube/Downloads/itineraries_csv'
output_file = '/Users/thusondube/Downloads/merged_output.parquet'
merge_csv_files_from_zips_to_parquet(folder_path, output_file)


In [4]:
import pandas as pd
import numpy as np 
import os
import zipfile


# Set the path to the root directory containing the zip files
root_dir = '/Users/thusondube/Downloads/itineraries_csv'

# Initialize an empty list to hold DataFrames
df_list = []

# Walk through the root directory
for folder_name, subfolders, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.zip'):
            # Get the full path to the zip file
            zip_file_path = os.path.join(folder_name, filename)
            
            # Open the zip file
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                # Loop through files inside the zip and check if it's a CSV
                for file in zip_ref.namelist():
                    if file.endswith('.csv'):
                        # Read the CSV into a DataFrame
                        with zip_ref.open(file) as f:
                            df = pd.read_csv(f)
                            # Append the DataFrame to the list
                            df_list.append(df)

# Concatenate all DataFrames into one
merged_df = pd.concat(df_list, ignore_index=True)

In [8]:
merged_df.head(1000000)

,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,e1b137527b9175d7d930c3af82e70ae0,2022-04-19,2022-05-20,OAK,ATL,PT7H52M,False,False,False,103.98,...,1653107460||1653126600,2022-05-20T22:31:00.000-06:00||2022-05-21T05:5...,DEN||ATL,OAK||DEN,Frontier Airlines||Frontier Airlines,F9||F9,||Airbus A320,9180||10620,943||1207,coach||coach
1,d813ebd107e3fa700206c0d96015da7a,2022-04-19,2022-05-20,OAK,ATL,PT6H15M,False,False,False,216.58,...,1653067080||1653084660,2022-05-20T10:18:00.000-07:00||2022-05-20T18:1...,LAX||ATL,OAK||LAX,Spirit Airlines||Spirit Airlines,NK||NK,||AIRBUS INDUSTRIE A320 SHARKLETS,4920||15600,None||None,coach||coach
2,e8ece5ad6f5962c696e06e031fc2a24a,2022-04-19,2022-05-20,OAK,ATL,PT9H6M,False,False,False,216.58,...,1653056820||1653084660,2022-05-20T07:27:00.000-07:00||2022-05-20T18:1...,LAX||ATL,OAK||LAX,Spirit Airlines||Spirit Airlines,NK||NK,AIRBUS INDUSTRIE A320 SHARKLETS||AIRBUS INDUST...,4920||15600,None||None,coach||coach
3,c004a54681335100f326c9613b3c9448,2022-04-19,2022-05-20,OAK,ATL,PT6H17M,False,False,False,237.58,...,1653110940||1653127980,2022-05-20T22:29:00.000-07:00||2022-05-21T06:1...,LAS||ATL,OAK||LAS,Spirit Airlines||Spirit Airlines,NK||NK,AIRBUS INDUSTRIE A320 SHARKLETS||Airbus A319,5580||13980,None||None,coach||coach
4,4a42bbf77211b4afa7b9e14005949120,2022-04-19,2022-05-20,OAK,ATL,PT14H12M,False,False,False,307.21,...,1653115560||1653159180,2022-05-20T23:46:00.000-07:00||2022-05-21T14:5...,SEA||ATL,OAK||SEA,Alaska Airlines||Alaska Airlines,AS||AS,Boeing 737-900||Boeing 737-900,7500||17580,672||2178,coach||coach
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,3db73913d6456af29f8d3a14cf3a5a32,2022-05-05,2022-06-29,DEN,LAX,PT4H33M,False,False,False,470.60,...,1656557160||1656567420,2022-06-29T20:46:00.000-06:00||2022-06-29T22:3...,SLC||LAX,DEN||SLC,United||Alaska Airlines,UA||AS,Embraer 175 (Enhanced Winglets)||Embraer 175,6120||7320,380||590,coach||coach
999996,cbbc46342387fe5aeae12d862be63b04,2022-05-05,2022-06-29,DEN,LAX,PT13H39M,False,False,False,520.11,...,1656518700||1656541800||1656563340,2022-06-29T10:05:00.000-06:00||2022-06-29T15:3...,CEZ||PHX||ONT,DEN||CEZ||PHX,Boutique Air||Boutique Air||American Airlines,4B||4B||AA,Pilatus PC-12||Pilatus PC-12||Canadair Regiona...,4500||5400||4620,265||329||347,coach||coach||coach
999997,6f45a7dd496ff123e979cba6afc984e3,2022-05-05,2022-06-29,DEN,LGA,PT3H50M,False,False,True,173.60,...,1656532500,2022-06-29T15:55:00.000-04:00,LGA,DEN,JetBlue Airways,B6,Airbus A320,13800,1621,coach
999998,77aa8436fd69662e8131d5a48b6e962c,2022-05-05,2022-06-29,DEN,LGA,PT3H51M,False,False,True,173.60,...,1656547440,2022-06-29T20:04:00.000-04:00,LGA,DEN,JetBlue Airways,B6,Airbus A320,13860,1621,coach


In [9]:
Sample_flights = merged_df.head(100000)


In [10]:
Sample_flights.to_csv('/Users/thusondube/Downloads/sample_flights.csv')

In [16]:
distinct_values_counts = {col: merged_df[col].nunique() for col in merged_df.columns}

In [17]:
for col, values in distinct_values_counts.items():
    print(f"Column '{col}' has distinct values and their counts:\n{values}\n")

Column 'legId' has distinct values and their counts:
1721518

Column 'searchDate' has distinct values and their counts:
32

Column 'flightDate' has distinct values and their counts:
92

Column 'startingAirport' has distinct values and their counts:
16

Column 'destinationAirport' has distinct values and their counts:
16

Column 'travelDuration' has distinct values and their counts:
1836

Column 'isBasicEconomy' has distinct values and their counts:
2

Column 'isRefundable' has distinct values and their counts:
2

Column 'isNonStop' has distinct values and their counts:
2

Column 'totalFare' has distinct values and their counts:
36175

Column 'totalTravelDistance' has distinct values and their counts:
2643

Column 'segmentsDepartureTimeEpochSeconds' has distinct values and their counts:
1273965

Column 'segmentsDepartureTimeRaw' has distinct values and their counts:
1374911

Column 'segmentsArrivalTimeEpochSeconds' has distinct values and their counts:
1447133

Column 'segmentsArrivalTi